In [2]:
from collections import deque
import copy
import numpy as np
import math
import random

class HexState():

	def __init__(self, board, player, move=None, turn=0, size=None): #fjernet move, kan legges til
		if size is not None:
			self.size = size
			self.board = [[0]*size]*size
		else:
			self.size = len(board)
			self.board = board
		self.current_player = player
		self.turn = turn
		self.move = copy.copy(move)

	def check_full_board(self):
		"""
    	Checks if a 2D list is filled with no 0s and that the number of 1s and -1s are equal.
    	"""
		for row in self.board:
			for element in row:
				if element == 0:
					return False
		return True


	def get_valid_moves(self):
		valid_cells = []
		for r in range(len(self.board)):
			for c in range(len(self.board)):
				if self.board[r][c] == 0:
					valid_cells.append((r, c))
		return valid_cells
	

	def execute_move(self, move):
		self.move = copy.copy(move)
		r = move[0]
		c = move[1]
		if self.board[r][c] != 0:
			raise Exception('ILLEGAL MOVE!')
		self.board[r][c] = self.next_player()

	
	def next_player(self):
		if self.current_player == 1:
			return 2
		elif self.current_player == 2:
			return 1


	def generate_child_states(self):
		child_states = []
		valid_moves = self.get_valid_moves()
		for move in valid_moves:
			board_copy = copy.deepcopy(self.board)
			child_state = HexState(board_copy, self.next_player(), move=move, turn=self.turn+1)
			child_state.execute_move(move)
			child_states.append(child_state)
		return child_states
		

	def get_winner(self):
		if self.turn < 2 * self.size - 1:
			return 0
		matrix = copy.deepcopy(self.board)
		queue = deque()
		visited = set()
		max_turns = self.size**2
		# Check if there is a path from right to left
		if self.current_player == 2 or self.turn == max_turns:
			for r in range(self.size):
				if matrix[r][0] == 1:
					queue.append((r,0))
					visited.add((r,0))
			while queue:
				r, c = queue.popleft()
				if c == len(matrix) - 1:
					return 1
				for x, y in ((r-1,c), (r,c-1), (r+1,c), (r,c+1), (r-1,c+1), (r+1,c-1)):
					if 0 <= x < len(matrix) and 0 <= y < len(matrix[0]) and matrix[x][y] == 1 and (x,y) not in visited:
						queue.append((x,y))
						visited.add((x,y))
		if self.current_player == 1 or self.turn == max_turns:
			# Check if there is a path from top to bottom
			for c in range(self.size):
				if matrix[0][c] == 2:
					queue.append((0,c))
					visited.add((0,c))
			while queue:
				r, c = queue.popleft()
				if r == len(matrix) - 1:
					return 2
				for x, y in ((r-1,c), (r,c-1), (r+1,c), (r,c+1), (r-1,c+1), (r+1,c-1)):
					if 0 <= x < len(matrix) and 0 <= y < len(matrix[0]) and matrix[x][y] == 2 and (x,y) not in visited:
						queue.append((x,y))
						visited.add((x,y))
		# Return 0 if no winner yet
		return 0


if __name__ == "__main__":
	board = [[0, 0, 0, 0, 2],
		 [2, 2, 2, 2, 2],
		 [1, 1, 1, 2, 1],
		 [0, 0, 0, 2, 0],
		 [0, 0, 0, 0, 0]]
         
	state = HexState(board, 1, turn=14)
	print(state.get_winner())

0


In [3]:
class Node:

    def __init__(self, state, parent=None):
        self.parent = parent
        self.state = state
        self.children = []
        self.wins = 0
        self.visits = 0

    def get_children(self):
        children = []
        for state in self.state.generate_child_states():
            child = Node(state, parent=self)
            children.append(child)
        return children

    def get_random_child(self):
        if self.children:
            return random.choice(self.children)
        else:
            return random.choice(self.get_children())

    def expand(self):
        if not self.children:
            self.children = self.get_children()
            
    def get_distribution(self):
        if not self.children:
            raise Exception('Node has no children')

        distribution = {}
        visits_sum = sum(child.visits for child in self.children)
        print(visits_sum)
        for child in self.children:
            distribution[child.state.move] = child.visits / visits_sum
        return distribution
    

    def get_list_distribution(self):
        dist = [[0 for i in range(self.state.size)] for j in range(self.state.size)]
        visits_sum = sum(child.visits for child in self.children)
        for child in self.children:
            move = child.state.move
            dist[move[0]][move[1]] = child.visits / visits_sum
        return dist
                

In [4]:
class MCTS:
    
    def __init__(self, params):
        self.params = params

    def select_action(self, node, starting_player, is_random=False):
        self.update(node, self.params['num_simulations'], starting_player)  # Update the tree using mcts

        if is_random:
            return random.choice(node.children)

        current_player = node.state.current_player
        action_node = None

        highest_qsa = -float('inf')
        lowest_qsa = float('inf')

        for child in node.children:
            qsa = float(child.wins)/float(child.visits)  # Calculate Q(s,a)

            if starting_player == current_player:
                if qsa > highest_qsa:
                    highest_qsa = qsa
                    action_node = child
            else:  # If the current player is the opposing player, the best score is the lowest Q(s,a)
                if qsa < lowest_qsa:
                    lowest_qsa = qsa
                    action_node = child
        return action_node

    def update(self, node, num_simulations, current_player):
        for _ in range(num_simulations):
            best_node = self.tree_search(node, current_player)
            best_node.expand()
            if len(best_node.get_children()) > 0:  # Choose a random child if just expanded
                best_node = random.choice(best_node.children)
            winner = self.evaluate(best_node)
            self.backpropagate(best_node, winner, current_player)

    def tree_policy_value(self, parent, child, is_opponent):
        q_value = child.wins / (child.visits + 1)
        u_value = self.params['C'] * math.sqrt(math.log(parent.visits) / (child.visits + 1))

        if is_opponent:
            return q_value - u_value
        return q_value + u_value

    def tree_search(self, node, current_player):
        if not node.children:  # Breaks recursion and returns the best leaf node
            return node

        best_child = node
        highest_value = float('-inf')
        lowest_value = float('inf')
        opposing_player = node.state.current_player != current_player

        for child in node.children:
            value = self.tree_policy_value(node, child, opposing_player)  # Get value of node based on the tree policy

            if opposing_player and value < lowest_value:
                # The best value is the lowest value when the player is the opposing player
                best_child = child
                lowest_value = value

            elif (not opposing_player) and value > highest_value:
                best_child = child
                highest_value = value
        return self.tree_search(best_child, current_player)  # Recursively search the tree until reaching best leaf node

    @staticmethod
    def evaluate(node):
        winner = node.state.get_winner()
        while winner == 0: #Kan optimaliseres
            node = node.get_random_child()
            winner = node.state.get_winner()
        return winner

    @staticmethod
    def backpropagate(node, winner, current_player):
        while node is not None:
            if winner == current_player:
                node.wins += 1
            node.visits += 1
            node = node.parent

In [5]:
def str_board(board):
    for row in board:
        print(row)
    print()

In [7]:
board = [[0, 0, 0, 0, 0, 0, 0], 
 		 [0, 0, 0, 0, 0, 0, 0], 
 		 [0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0]]

"""
board = [[0, 0, 0, 0, 0], 
 		 [0, 0, 0, 0, 0], 
 		 [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]
"""

params = {
	'num_simulations': 1500,
	'C': 1.4
}
mcts = MCTS(params)
results = []

starting_player = 1

board_data = []
player_data = []
distribution_data = []

tuple_data = []

for i in range(30):
    
    if starting_player == 1:
        s0 = HexState(board, 1)
        starting_player = 2
    else:
        s0 = HexState(board, 2)
        starting_player = 1
    
    node = Node(s0)
    turn = 0


    while node.state.get_winner() == 0:
        print(f'Game {i}, turn {turn}')
        #root = node
        turn += 1
        if node.state.current_player == 1:
            node = mcts.select_action(node, node.state.current_player)
        elif node.state.current_player == 2:
            node = mcts.select_action(node, node.state.current_player)
        print(node.state.get_winner())
        #data.append([node.parent.state.board, node.parent.get_list_distribution(), node.parent.state.current_player])
        data_tuple = sum(node.parent.state.board, [])
        data_tuple.insert(0, node.parent.state.current_player)
        tuple_data.append(data_tuple)
        #board_data.append(sum(node.parent.state.board, []))
        board_data.append(node.parent.state.board)
        player_data.append(node.parent.state.current_player)
        distribution_data.append(sum(node.parent.get_list_distribution(), []))

Game 0, turn 0
0
Game 0, turn 1
0
Game 0, turn 2
0
Game 0, turn 3
0
Game 0, turn 4
0
Game 0, turn 5
0
Game 0, turn 6
0
Game 0, turn 7
0
Game 0, turn 8
0
Game 0, turn 9
0
Game 0, turn 10
0
Game 0, turn 11
0
Game 0, turn 12
1
Game 1, turn 0
0
Game 1, turn 1
0
Game 1, turn 2
0
Game 1, turn 3
0
Game 1, turn 4
0
Game 1, turn 5
0
Game 1, turn 6
0
Game 1, turn 7
0
Game 1, turn 8
0
Game 1, turn 9
0
Game 1, turn 10
0
Game 1, turn 11
0
Game 1, turn 12
0
Game 1, turn 13
0
Game 1, turn 14
0
Game 1, turn 15
0
Game 1, turn 16
0
Game 1, turn 17
0
Game 1, turn 18
2
Game 2, turn 0
0
Game 2, turn 1
0
Game 2, turn 2
0
Game 2, turn 3
0
Game 2, turn 4
0
Game 2, turn 5
0
Game 2, turn 6
0
Game 2, turn 7
0
Game 2, turn 8
0
Game 2, turn 9
0
Game 2, turn 10
0
Game 2, turn 11
0
Game 2, turn 12
0
Game 2, turn 13
0
Game 2, turn 14
0
Game 2, turn 15
2
Game 3, turn 0
0
Game 3, turn 1
0
Game 3, turn 2
0
Game 3, turn 3
0
Game 3, turn 4
0
Game 3, turn 5
0
Game 3, turn 6
0
Game 3, turn 7
0
Game 3, turn 8
0
Game 3, turn 

# Husk å lagre dataene som CSV

In [8]:
tuple_data[3]

[2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [9]:
def one_hot_boards(lst):
    """Takes a list of board states and replaces 0s, 1s and 2s with
    [0, 0], [1, 0] and [0, 1]"""
    new_lst = []
    for board in lst:
        new_board = []
        for row in board:
            new_row = []
            for i in range(len(row)):
                if row[i] == 0:
                    new_row.append([0, 0])
                elif row[i] == 1:
                    new_row.append([1, 0])
                elif row[i] == 2:
                    new_row.append([0, 1])
            new_board.append(new_row)
        new_lst.append(new_board)
    return new_lst
    

In [10]:
def one_hot_tuple(lst):
    new_lst = []
    for i in range(len(lst)):
        row = []
        for j in range(len(lst[0])):
            if lst[i][j] == 0:
                row.append([0, 0])
            elif lst[i][j] == 1:
                row.append([1, 0])
            elif lst[i][j] == 2:
                row.append([0, 1])
        new_lst.append(row)
    return new_lst

In [11]:
def one_hot_pids(lst):
    new_lst = []
    for i in range(len(lst)):
        if lst[i] == 1:
            new_lst.append(np.array([1, 0]))
        elif lst[i] == 2:
            new_lst.append(np.array([0, 1]))
    return new_lst

In [12]:
def lists_to_arrays(lst):
    new_lst = []
    for i in range(len(lst)):
        new_lst.append(np.array(lst[i]))
    return new_lst

In [ ]:
np.array([[[0, 1], [1, 0]], [[0, 0], [1, 0]]])

array([[[0, 1],
        [1, 0]],

       [[0, 0],
        [1, 0]]])

In [59]:
import csv

# open a CSV file in write mode
with open('player77.txt', 'w', newline='') as file:
    writer = csv.writer(file)

    # write the list as a single row in the CSV file
    writer.writerow(player_data)

In [65]:
import csv

# open a CSV file in write mode
with open('tuple77.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # write each sublist as a row in the CSV file
    for sublist in tuple_data:
        writer.writerow(sublist)

In [13]:
#training_boards = lists_to_arrays(one_hot_boards(board_data))
#training_pids = lists_to_arrays(one_hot_pids(player_data))
#training_dists = lists_to_arrays(distribution_data)

training_boards = np.array(one_hot_boards(board_data))
training_pids = np.array(one_hot_pids(player_data))
training_dists = np.array(distribution_data)
training_tuple = np.array(one_hot_tuple(tuple_data))

### We gooooo!

In [28]:
import keras
from keras.layers import Input, Conv2D, Flatten, Dense, concatenate
from keras.models import Model

# define the shape of the board state
input_shape = (7, 7, 2)

# create two input layers for the board state and player to move
input_board = Input(shape=input_shape, name='input_board')
input_player = Input(shape=(2,), name='input_player')

# create the convolutional layers for the board state
conv1 = Conv2D(filters=48, kernel_size=3, activation='relu')(input_board) # Var 48
conv2 = Conv2D(filters=96, kernel_size=3, activation='relu')(conv1) # Var 96

# flatten the output from the convolutional layers
flatten = Flatten()(conv2)

# concatenate the flattened board state and player to move
concat = concatenate([flatten, input_player])

# add a dense layer
dense1 = Dense(units=128, activation='relu')(concat) # Var 128
dense2 = Dense(units=160, activation='relu')(dense1) # Var 160
#dense3 = Dense(units=2048, activation='relu')(dense2)

# add the output layer
output = Dense(units=7*7, activation='softmax')(dense2)

# create the model with multiple inputs
model = Model(inputs=[input_board, input_player], outputs=output)

# compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.0001) # Var 0.0001
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
training_boards.shape

(13189, 5, 5, 2)

In [ ]:
np.size(training_boards)

659450

In [30]:
X_2d = training_boards
X_scalar = training_pids
y = training_dists

# train the model
model.fit([X_2d, X_scalar], y, epochs=10, batch_size=1)

Epoch 1/10
601/601 [==============================] - 2s 3ms/step - loss: 3.7513 - accuracy: 0.0116
Epoch 2/10
601/601 [==============================] - 2s 3ms/step - loss: 3.7373 - accuracy: 0.0233
Epoch 3/10
601/601 [==============================] - 2s 3ms/step - loss: 3.7278 - accuracy: 0.0283
Epoch 4/10
601/601 [==============================] - 2s 3ms/step - loss: 3.7228 - accuracy: 0.0283
Epoch 5/10
601/601 [==============================] - 2s 3ms/step - loss: 3.7180 - accuracy: 0.0349
Epoch 6/10
601/601 [==============================] - 2s 3ms/step - loss: 3.7130 - accuracy: 0.0316
Epoch 7/10
601/601 [==============================] - 2s 3ms/step - loss: 3.7085 - accuracy: 0.0383
Epoch 8/10
601/601 [==============================] - 2s 3ms/step - loss: 3.7055 - accuracy: 0.0466
Epoch 9/10
601/601 [==============================] - 2s 3ms/step - loss: 3.7031 - accuracy: 0.0532
Epoch 10/10
601/601 [==============================] - 2s 3ms/step - loss: 3.6991 - accuracy: 0.0616

In [31]:
input_1 = np.array([[[[0, 0], [0, 1], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]],
                    [[0, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0]],
                    [[0, 0], [0, 0], [0, 0], [0, 1], [0, 0], [0, 0], [0, 0]],
                    [[0, 1], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]],
                    [[0, 0], [0, 0], [0, 0], [0, 1], [0, 0], [0, 0], [0, 0]],
                    [[0, 1], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]],
                    [[0, 0], [0, 0], [0, 1], [0, 0], [0, 0], [0, 0], [0, 0]]]])

print(input_1.shape)

input_2 = np.array([[1, 0]])

# Get the model's prediction for the single sample
prediction = model.predict([input_1, input_2])

# Print the prediction
print(prediction)

(1, 7, 7, 2)
1/1 [==============================] - 0s 121ms/step
[[0.02274751 0.01799927 0.03304184 0.02221362 0.00822669 0.01989548
  0.04317848 0.02122386 0.01005032 0.00261117 0.00159461 0.00348375
  0.0145644  0.0043767  0.01938518 0.01937575 0.02757671 0.00188653
  0.03289103 0.03640017 0.01622875 0.00670065 0.03531274 0.02451658
  0.03131622 0.04233383 0.0220207  0.02291975 0.02314774 0.03559517
  0.01592152 0.00284935 0.01494281 0.01739857 0.05516557 0.00971278
  0.01968862 0.01041764 0.01903347 0.01472786 0.01541299 0.02018301
  0.01920976 0.02647581 0.01162556 0.02400468 0.0319647  0.01792762
  0.0305224 ]]


In [32]:
def rescale_distribution(prediction, case):
    rescaled = []
    board = case[0][:]
    
    counter = 0
    for row in board:
        for element in row:
            if element[0] != 0 or element[1] != 0:
                rescaled.append(0)
            else:
                rescaled.append(prediction[0][counter])
            counter += 1

    total = sum(rescaled)
    dist = [x / total for x in rescaled]
    return dist

rescale_distribution(prediction, input_1)

[0.024927556842813395,
 0.0,
 0.03620845958177061,
 0.02434249658944227,
 0.00901510671961486,
 0.021802199962860987,
 0.04731655816400282,
 0.02325788005299544,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.02124299093373888,
 0.021232650444039273,
 0.030219572899585403,
 0.0,
 0.03604319136819029,
 0.03988864109055401,
 0.017784056306125957,
 0.0,
 0.03869699456636761,
 0.026866163928699685,
 0.03431745630574896,
 0.04639095982545131,
 0.02413108399160632,
 0.02511629629426497,
 0.02536614383506255,
 0.039006490770902194,
 0.01744738416794341,
 0.0,
 0.01637488188462867,
 0.019065991267219674,
 0.060452454127801175,
 0.0,
 0.021575507280956488,
 0.011416033303422899,
 0.020857571939060866,
 0.0161393234472938,
 0.016890116276939017,
 0.022117276684339653,
 0.021050757025444487,
 0.02901315658000493,
 0.0,
 0.02630520154769208,
 0.03502808839613546,
 0.019645742477221537,
 0.03344756312005808]

### Prøv å optimere modellen

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val = train_test_split(X_2d, test_size=0.2, random_state=42)
x_train_2, x_val_2 = train_test_split(X_scalar, test_size=0.2, random_state=42)
y_train, y_val = train_test_split(y, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from kerastuner.tuners import RandomSearch

# Define the model-building function for the tuner
def build_model(hp):
    # Define the model architecture with hyperparameters to tune
    input_1 = layers.Input(shape=(5, 5, 2))
    conv_1 = layers.Conv2D(filters=hp.Int('conv_1_filters', min_value=16, max_value=64, step=16), kernel_size=3, activation='relu')(input_1)
    conv_2 = layers.Conv2D(filters=hp.Int('conv_2_filters', min_value=32, max_value=128, step=32), kernel_size=3, activation='relu')(conv_1)
    flatten = layers.Flatten()(conv_2)
    input_2 = layers.Input(shape=(2,))
    concat = layers.concatenate([flatten, input_2])
    dense1 = layers.Dense(units=hp.Int('dense_units1', min_value=32, max_value=256, step=32), activation='relu')(concat)
    #dense2 = layers.Dense(units=hp.Int('dense_units2', min_value=32, max_value=256, step=32), activation='relu')(dense1)
    #dense3 = layers.Dense(units=hp.Int('dense_units3', min_value=32, max_value=256, step=32), activation='relu')(dense2)
    output = layers.Dense(units=25, activation='softmax')(dense1)

    model = keras.models.Model(inputs=[input_1, input_2], outputs=output)

    # Compile the model with hyperparameters to tune
    optimizer = keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define the search space for hyperparameters
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=3,
    directory='dir7',
    project_name='my_project7')

# Search for the best hyperparameters using cross-validation
tuner.search(x=[x_train, x_train_2], y=y_train, epochs=10, batch_size=8, validation_data=([x_val, x_val_2], y_val))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Train the model with the best hyperparameters on all the training data
model = tuner.hypermodel.build(best_hps)
model.fit([x_train, x_train_2], y_train, epochs=10, batch_size=32)

Trial 10 Complete [00h 02m 42s]
val_accuracy: 0.09893859177827835

Best val_accuracy So Far: 0.25372756520907086
Total elapsed time: 00h 26m 29s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
330/330 [==============================] - 3s 5ms/step - loss: 3.0247 - accuracy: 0.0878
Epoch 2/10
330/330 [==============================] - 2s 6ms/step - loss: 2.9129 - accuracy: 0.1607
Epoch 3/10
330/330 [==============================] - 2s 6ms/step - loss: 2.8994 - accuracy: 0.1876
Epoch 4/10
330/330 [==============================] - 2s 5ms/step - loss: 2.8921 - accuracy: 0.2166
Epoch 5/10
330/330 [==============================] - 2s 6ms/step - loss: 2.8878 - accuracy: 0.2340
Epoch 6/10
330/330 [==============================] - 2s 7ms/step - loss: 2.8853 - accuracy: 0.2416
Epoch 7/10
330/330 [==============================] - 2s 6ms/step - loss: 2.8819 - accuracy: 0.2639
Epoch 8/10
330/330 [==============================] - 2s 5ms/step - loss: 2.8800 - accuracy: 0.2620
Epoch 9/10
330/33

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best filter size 1:", best_hps.get('conv_1_filters'))
print("Best filter size 2:", best_hps.get('conv_2_filters'))
print("Best dense units 1:", best_hps.get('dense_units1'))
#print("Best dense units 2:", best_hps.get('dense_units2'))
#print("Best dense units 3:", best_hps.get('dense_units3'))
print("Best learning rate:", best_hps.get('learning_rate'))

Best filter size 1: 64
Best filter size 2: 96
Best dense units 1: 160
Best learning rate: 0.001


### Prøva seg med litt tuple...

In [35]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

# define the input shape
input_shape = (50, 2)

# create the model
model2 = Sequential([
    Flatten(input_shape=input_shape),
    Dense(1024, activation='relu'),
    Dense(1024, activation='relu'),
    Dense(1024, activation='relu'),
    Dense(49, activation='softmax')
])

# compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# print the model summary
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_8 (Flatten)         (None, 100)               0         
                                                                 
 dense_30 (Dense)            (None, 1024)              103424    
                                                                 
 dense_31 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_32 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_33 (Dense)            (None, 49)                50225     
                                                                 
Total params: 2,252,849
Trainable params: 2,252,849
Non-trainable params: 0
_________________________________________________________________


In [ ]:
training_tuple[0]

array([[1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]])

In [37]:
X_tuple = training_tuple
y = training_dists

# train the model
model2.fit(X_tuple, y, epochs=10, batch_size=1)

Epoch 1/10
601/601 [==============================] - 6s 10ms/step - loss: 3.7352 - accuracy: 0.0150
Epoch 2/10
601/601 [==============================] - 6s 9ms/step - loss: 3.7079 - accuracy: 0.0366
Epoch 3/10
601/601 [==============================] - 6s 9ms/step - loss: 3.6994 - accuracy: 0.0499
Epoch 4/10
601/601 [==============================] - 6s 10ms/step - loss: 3.6930 - accuracy: 0.0566
Epoch 5/10
601/601 [==============================] - 7s 12ms/step - loss: 3.6848 - accuracy: 0.0566
Epoch 6/10
601/601 [==============================] - 10s 17ms/step - loss: 3.6795 - accuracy: 0.0899
Epoch 7/10
601/601 [==============================] - 9s 15ms/step - loss: 3.6738 - accuracy: 0.0899
Epoch 8/10
601/601 [==============================] - 10s 16ms/step - loss: 3.6706 - accuracy: 0.1065
Epoch 9/10
601/601 [==============================] - 9s 15ms/step - loss: 3.6668 - accuracy: 0.1148
Epoch 10/10
601/601 [==============================] - 8s 13ms/step - loss: 3.6644 - accura

In [38]:
input_data = np.array([[[1, 0], 
                  [0, 0], [0, 1], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], 
                  [0, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], 
                  [0, 0], [0, 0], [0, 0], [0, 1], [0, 0], [0, 0], [0, 0], 
                  [0, 1], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0],
                  [0, 0], [0, 0], [0, 0], [0, 1], [0, 0], [0, 0], [0, 0], 
                  [0, 1], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0],
                  [0, 0], [0, 0], [0, 1], [0, 0], [0, 0], [0, 0], [0, 0],]])

# Get the model's prediction for the single sample
prediction = model2.predict(input_data)

# Print the prediction
print(prediction)

1/1 [==============================] - 0s 107ms/step
[[0.04873709 0.01328062 0.01626215 0.02852838 0.01012534 0.01859059
  0.05000911 0.02217293 0.00917549 0.00471084 0.00208148 0.00475581
  0.00151765 0.0014175  0.02779712 0.0605828  0.02057467 0.00494814
  0.00804908 0.05352195 0.01985833 0.00520361 0.01934408 0.01306715
  0.0147616  0.01743596 0.02068047 0.02148508 0.02200236 0.03445309
  0.00798924 0.00278352 0.02160016 0.00844242 0.04188304 0.01102509
  0.02641976 0.02824015 0.01291418 0.02750395 0.01421821 0.03016911
  0.01466735 0.02647411 0.02230774 0.04121975 0.02373685 0.01832406
  0.02495081]]


In [ ]:
prediction.shape

(1, 25)

In [ ]:
rescaled = rescale_distribution(prediction, input_data)
rescaled

IndexError: invalid index to scalar variable.

# Optimaliser tupel:

In [40]:
from sklearn.model_selection import train_test_split

x_train, x_val = train_test_split(training_tuple, test_size=0.2, random_state=42)
y_train, y_val = train_test_split(training_dists, test_size=0.2, random_state=42)

In [52]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from kerastuner.tuners import RandomSearch

# Define the model-building function for the tuner
def build_model(hp):
    # Define the model architecture with hyperparameters to tune
    input_shape = (50, 2)

# create the model
    model = Sequential([
        Flatten(input_shape=input_shape),
        Dense(hp.Int('dense1', min_value=512, max_value=2048, step=512), activation='relu'),
        Dense(hp.Int('dense2', min_value=512, max_value=2048, step=512), activation='relu'),
        Dense(hp.Int('dense3', min_value=512, max_value=2048, step=512), activation='relu'),
        Dense(49, activation='softmax')
    ])

    # Compile the model with hyperparameters to tune
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define the search space for hyperparameters
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=3,
    directory='dir17',
    project_name='my_project17')

# Search for the best hyperparameters using cross-validation
tuner.search(x=x_train, y=y_train, epochs=10, batch_size=8, validation_data=(x_val, y_val))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Train the model with the best hyperparameters on all the training data
model = tuner.hypermodel.build(best_hps)
model.fit(x_train, y_train, epochs=10, batch_size=8)

Trial 10 Complete [00h 00m 52s]
val_accuracy: 0.022038566569487255

Best val_accuracy So Far: 0.033057850475112595
Total elapsed time: 00h 15m 10s
INFO:tensorflow:Oracle triggered exit


Epoch 1/10
60/60 [==============================] - 3s 37ms/step - loss: 3.8678 - accuracy: 0.0125
Epoch 2/10
60/60 [==============================] - 2s 36ms/step - loss: 3.8138 - accuracy: 0.0125
Epoch 3/10
60/60 [==============================] - 2s 38ms/step - loss: 3.7692 - accuracy: 0.0292
Epoch 4/10
60/60 [==============================] - 2s 37ms/step - loss: 3.7429 - accuracy: 0.0375
Epoch 5/10
60/60 [==============================] - 2s 37ms/step - loss: 3.7270 - accuracy: 0.0292
Epoch 6/10
60/60 [==============================] - 2s 37ms/step - loss: 3.7144 - accuracy: 0.0521
Epoch 7/10
60/60 [==============================] - 2s 37ms/step - loss: 3.7057 - accuracy: 0.0625
Epoch 8/10
60/60 [==============================] - 2s 37ms/step - loss: 3.6976 - accuracy: 0.0688
Epoch 9/10
60/60 [==============================] - 2s 38ms/step - loss: 3.6931 - accuracy: 0.0646
Epoch 10/10
60/60 [==============================] - 2s 37ms/step - loss: 3.6866 - accuracy: 0.0854


In [54]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best dense units 1:", best_hps.get('dense1'))
print("Best dense units 2:", best_hps.get('dense2'))
print("Best dense units 3:", best_hps.get('dense3'))
#print("Best dense units 2:", best_hps.get('dense_units2'))
#print("Best dense units 3:", best_hps.get('dense_units3'))
#print("Best learning rate:", best_hps.get('learning_rate'))

Best dense units 1: 1024
Best dense units 2: 2048
Best dense units 3: 512


## Gammelt:

In [ ]:
def merge_arrays(pids, boards):
    """Takes two lists of equal length and joins the numpy arrays"""
    merged_lst = []
    for i in range(len(pids)):
        merged_array = np.vstack([pids[i], boards[i].reshape(-1)])
        merged_lst.append(merged_array)
    return merged_lst

In [ ]:
from keras.layers import Input, Conv2D, Flatten, Dense, concatenate
from keras.models import Model

# create two input layers for the 2D data and scalar value
input_2d = Input(shape=(7, 7, 2), name='input_2d')
input_scalar = Input(shape=(2), name='input_scalar')

# create the convolutional layers for the 2D data
conv1 = Conv2D(filters=16, kernel_size=3, activation='relu')(input_2d)
conv2 = Conv2D(filters=32, kernel_size=3, activation='relu')(conv1)

# flatten the output from the convolutional layers
flatten = Flatten()(conv2)

# concatenate the flattened 2D data and scalar value
concat = concatenate([flatten, input_scalar])

# add a dense layer
dense = Dense(units=64, activation='relu')(concat)

# add the output layer
output = Dense(units=7*7, activation='sigmoid')(dense)

# create the model with multiple inputs
model = Model(inputs=[input_2d, input_scalar], outputs=output)

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# print the model summary
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2d (InputLayer)          [(None, 7, 7, 2)]    0           []                               
                                                                                                  
 conv2d_20 (Conv2D)             (None, 5, 5, 16)     304         ['input_2d[0][0]']               
                                                                                                  
 conv2d_21 (Conv2D)             (None, 3, 3, 32)     4640        ['conv2d_20[0][0]']              
                                                                                                  
 flatten_18 (Flatten)           (None, 288)          0           ['conv2d_21[0][0]']              
                                                                                            

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense

class HexNet:
    def __init__(self, board_size):
        self.board_size = board_size
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(self.board_size, self.board_size, 2)))
        model.add(Conv2D(64, kernel_size=3, activation='relu'))
        model.add(Flatten())
        model.add(Dense(self.board_size**2, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam')
        return model

    def predict(self, x, player):
        # Convert the board state to a tensor and one-hot encode
        x = np.array(x).reshape(1, self.board_size, self.board_size, 1)
        x = np.concatenate((x == player, x == -player), axis=3)

        # Get prediction for each cell
        pred = self.model.predict(x)[0]

        return pred.reshape(self.board_size, self.board_size)